# 08 - 工具变量

## 绕过省略变量偏差（OVB）

控制OVB的一种方法是，将省略的变量添加到我们的模型中。但是，这并不总是可能的，主要是因为我们根本没有关于省略变量的数据。例如，让我们回到教育对工资影响的模型：

$
log（hwage）_i = beta_0 + kappa  educ_i + pmb{beta}Ability_i + u_i
$

为了弄清楚教育（kappa）对（log（hwage））的因果效应，我们需要控制能力因素（Ability_i）。如果我们不这样做，我们可能会有一些偏差，毕竟，能力可能是一个混淆因子，同时影响干预变量：教育和结果变量：收入。